In [1]:
from sklearn.datasets import load_iris
iris = load_iris()

In [2]:
import numpy as np
import pandas as pd
print ('your pandas version is: %s' % pd.__version__)

your pandas version is: 0.20.1


In [3]:
print ('Your numpy version is: %s' % np.__version__)

Your numpy version is: 1.12.1


In [4]:
iris_narray = iris.data
iris_dataframe = pd.DataFrame(iris.data, columns = iris.feature_names)

In [5]:
iris_dataframe['group'] = pd.Series([iris.target_names[k] for k in iris.target], dtype = 'category')

In [6]:
# using pandas you can quickly compute both mean and medians
print (iris_dataframe.mean(numeric_only = True))

sepal length (cm)    5.843333
sepal width (cm)     3.054000
petal length (cm)    3.758667
petal width (cm)     1.198667
dtype: float64


In [7]:
print (iris_dataframe.median(numeric_only = True))

sepal length (cm)    5.80
sepal width (cm)     3.00
petal length (cm)    4.35
petal width (cm)     1.30
dtype: float64


In [8]:
# When checking for central tendency measures, you should:
# 1. verify wheather means are zero
# 2. check whether they are different from each other.
# 3. Notice whether the median is different from the mean

In [9]:
# measuring variance and range
import math
square = math.sqrt
std = iris_dataframe.std()
variance = np.sqrt(std)
print ('standard deviation of iris_dataframe is: ',std)
print ('')
print ('variance of iris_dataframe is: ',variance)

standard deviation of iris_dataframe is:  sepal length (cm)    0.828066
sepal width (cm)     0.433594
petal length (cm)    1.764420
petal width (cm)     0.763161
dtype: float64

variance of iris_dataframe is:  sepal length (cm)    0.909981
sepal width (cm)     0.658479
petal length (cm)    1.328315
petal width (cm)     0.873591
dtype: float64


In [10]:
# The variance is a good indicator of wheather a mean is a suitable
# indicator of the variable distribution

In [11]:
# maximum and minimum value in our data
print (iris_dataframe.max(numeric_only = True)-iris_dataframe.min(numeric_only = True))

sepal length (cm)    3.6
sepal width (cm)     2.4
petal length (cm)    5.9
petal width (cm)     2.4
dtype: float64


In [12]:
# working with percentiles
print (iris_dataframe.quantile(np.array([0,.25,.5,.75])))

      sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0.00                4.3               2.0               1.00               0.1
0.25                5.1               2.8               1.60               0.3
0.50                5.8               3.0               4.35               1.3
0.75                6.4               3.3               5.10               1.8


In [13]:
# Skewness: defines the asymmetry of data with respect to the mean.
# if the skew is negative, the left tail is too long and the mass of the
# the observations are on the right side of the distribution. If it is 
# positive, it is excactly the opposite.

# Kurtosis: shows whether the data distribution, especially the peak
# and tails, are of the right shape. If the kurtosis is above zero, the 
# distribution has a marked peak. If it is below zero, the distribution is 
# is too flat instead

# when performing the kurtosis and skewness test, you determine whether
# the p-value is less than or equal to 0.05. If so, you have to reject normality.

In [14]:
from scipy.stats import kurtosis, kurtosistest
k = kurtosis(iris_dataframe['petal length (cm)'])
rscore, pvalue = kurtosistest(iris_dataframe['petal length (cm)'])
print ('Kurtosis %0.3f z-score %0.3f p-value %0.3f' %(k,rscore, pvalue))

Kurtosis -1.395 z-score -14.811 p-value 0.000


In [15]:
from scipy.stats import skew, skewtest
s = skew(iris_dataframe['petal length (cm)'])
rscore , pvalue = skewtest(iris_dataframe['petal length (cm)'])
print ('Skewness %0.3f z-score %0.3f p-value %0.3f' % (s, rscore, pvalue))

Skewness -0.272 z-score -1.398 p-value 0.162


In [20]:
# to divide into binss according to specific intervals, the pandas
# dataframe features two useful functions cut and qcut that can
# transfrom a metric variable into a qualitative one.

# cut:expects a series of edge values used to cut the measurement
# or an integer number of groups used to cut the variables into 
# equal-width bins.

# qcut: expects a series percentiles used to cut the variable

# check out binning and discretization
import pandas as pd
iris_binned = pd.concat([pd.qcut(iris_dataframe.ix[:,0], [0,0.25,.5,.75,1]),
                        pd.qcut(iris_dataframe.ix[:,1], [0,.25,.5,.75,1])
                        pd.qcut(iris_dataframe.ix[:,2], [0,.25,.5,.75,1])
                        pd.qcut(iris_dataframe.ix[:,3], [0,.25,.5,.75,1])
                        ], join = 'outer', axis = 1)

SyntaxError: invalid syntax (<ipython-input-20-cbc4f94feb8c>, line 15)

In [21]:
boxplots = iris_dataframe.boxplot(return_type = 'axes')

In [25]:
boxplots.add_image


<bound method _AxesBase.add_image of <matplotlib.axes._subplots.AxesSubplot object at 0x000000000A38C2B0>>

In [26]:
# performing t-tests after boxplots
from scipy.stats import ttest_ind
group0 = iris_dataframe['group'] == 'setosa'
group1 = iris_dataframe['group'] == 'versicolor'
group2 = iris_dataframe['group'] == 'virginica'
print ('var1 %0.3f var2 %03f' % (iris_dataframe['petal length (cm)'].var(),
                                iris_dataframe['petal length (cm)'].var()))


var1 3.113 var2 3.113179


In [27]:
# calculating variance 
t, pvalue= ttest_ind(iris_dataframe['sepal width (cm)'][group1],
                    iris_dataframe['sepal width (cm)'][group2], axis = 0, equal_var = False)
print ('t statistics %0.3f p-value %0.3f' %(t, pvalue))

t statistics -3.206 p-value 0.002


In [29]:
from scipy.stats import f_oneway
f, pvalue = f_oneway(iris_dataframe['sepal width (cm)'][group0],
                    iris_dataframe['sepal width (cm)'][group1],
                    iris_dataframe['sepal width (cm)'][group2])
print ("one-way ANOVA F-value %0.3f p-value %0.3f" % (f, pvalue))

one-way ANOVA F-value 47.364 p-value 0.000


In [30]:
from pandas.tools.plotting import parallel_coordinates
iris_dataframe['labels'] = [iris.target_names[k] for k in iris_dataframe['group']]
pll = parallel_coordinates(iris_dataframe, 'labels')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices